In [1]:
import numpy as np
from sklearn.datasets import load_digits
import tensorflow as tf
from tensorflow.keras import layers
!pip install tinymlgen
from tinymlgen import port

  Created wheel for tinymlgen: filename=tinymlgen-0.2-cp37-none-any.whl size=2243 sha256=108e0b710eb3d0583286eaa7cdd2ac6715cfebc2cfb3baf7022c4d6b7aedb94f
  Stored in directory: /root/.cache/pip/wheels/a1/84/4d/3c1a67c7b9483e296ff1b2f4a13e2f800ca2a6093b741450d2
  Created wheel for hexdump: filename=hexdump-3.3-cp37-none-any.whl size=8913 sha256=a34b7c48d0ea868c73881909b25b88e78959c85521cb9cd8d5d8ce54f8bd71c1
  Stored in directory: /root/.cache/pip/wheels/d5/d1/f2/c8183b5863b3df595c2eeafd8e015a43dae13d403a959467c6
Successfully built tinymlgen hexdump


In [2]:
def get_data():
    np.random.seed(1337)
    x_values, y_values = load_digits(return_X_y=True)
    x_values /= x_values.max()
    # reshape to (8 x 8 x 1)
    x_values = x_values.reshape((len(x_values), 8, 8, 1))

    # split into train, validation, test
    TRAIN_SPLIT = int(0.6 * len(x_values))
    TEST_SPLIT = int(0.2 * len(x_values) + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

    return x_train, x_test, x_validate, y_train, y_test, y_validate

In [3]:
def get_model():
    x_train, x_test, x_validate, y_train, y_test, y_validate = get_data()

    # create a CNN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(8, 8, 1)))
    # model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(len(np.unique(y_train))))

    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=50, batch_size=16,
                        validation_data=(x_validate, y_validate))
    return model, x_test, y_test

In [4]:
def test_model(model, x_test, y_test):
    x_test = (x_test / x_test.max()).reshape((len(x_test), 8, 8, 1))
    y_pred = model.predict(x_test).argmax(axis=1)

    print('ACCURACY', (y_pred == y_test).sum() / len(y_test))

In [5]:
if __name__ == '__main__':
    model, x_test, y_test = get_model()
    test_model(model, x_test, y_test)
    c_code = port(model, variable_name='digits_model', pretty_print=True)
    print(c_code)

Epoch 1/50
68/68 [==============================] - 31s 5ms/step - loss: 2.0314 - accuracy: 0.4416 - val_loss: 1.6653 - val_accuracy: 0.7472
Epoch 2/50
68/68 [==============================] - 0s 3ms/step - loss: 1.1985 - accuracy: 0.8655 - val_loss: 0.9571 - val_accuracy: 0.8139
Epoch 3/50
68/68 [==============================] - 0s 3ms/step - loss: 0.6169 - accuracy: 0.9193 - val_loss: 0.6683 - val_accuracy: 0.8528
Epoch 4/50
68/68 [==============================] - 0s 3ms/step - loss: 0.3896 - accuracy: 0.9314 - val_loss: 0.5604 - val_accuracy: 0.8556
Epoch 5/50
68/68 [==============================] - 0s 3ms/step - loss: 0.2919 - accuracy: 0.9388 - val_loss: 0.5071 - val_accuracy: 0.8528
Epoch 6/50
68/68 [==============================] - 0s 3ms/step - loss: 0.2281 - accuracy: 0.9471 - val_loss: 0.4715 - val_accuracy: 0.8611
Epoch 7/50
68/68 [==============================] - 0s 3ms/step - loss: 0.1891 - accuracy: 0.9555 - val_loss: 0.4455 - val_accuracy: 0.8667
Epoch 8/50
68/68 [=

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()


# Save the model.
with open('model_quantized.tflite', 'wb') as f:
  f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpbu9j9nj_/assets


INFO:tensorflow:Assets written to: /tmp/tmpbu9j9nj_/assets


In [7]:
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [8]:
# Write TFLite model to a C source (or header) file
with open("quant_model.h", 'w') as file:
  file.write(hex_to_c_array(quantized_tflite_model, "quant_8_model"))